# COMP8240 : Project Group K

In [1]:
#pip install tweets

## Code for Collecting Tweets

In [2]:
import tweepy
import json
import pandas as pd
import numpy as np
import sys

#### Setting Up Access to Twitter API

In [3]:
twitter_keys = {
  'consumer_key': '7UmkNb6j9Fv8MdckV9kcmpWex',
  'consumer_secret': '6fDqb0S670vgNktrMLS3awceMqGCDMn2dXxkRYhNOZ04YJ63vz',
  'access_token_key': '1299904370494992384-0XzXQq55hgrxZ3mlUAKqgP5e5N6Gx8',
  'access_token_secret': '2SXqY97u8bT1Q91rLvZUwmS3GvsNkdtBhgn7iQ8xNuDal'
}

In [4]:
# Setup access to API
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'],twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'],twitter_keys['access_token_secret'])
api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


In [5]:
#  Testing the authentication.
try:
  user = api.verify_credentials()
  print('Authentication OK.')
except:
  print('Error during authentication.')

Authentication OK.


In [6]:
# The name of the user.
print('The name of the user is: ' + str(user.name) + '.')
# The location of the user.
print('The location of the user is: ' + str(user.location) + '.')
# The number of followers this user has.
print('The user has ' + str(user.followers_count) + ' followers.')
# The number of friends who this user has.
print('The user has ' + str(user.friends_count) + ' friends.') 

The name of the user is: Vikram Rajan.
The location of the user is: Macquarie Park, Sydney.
The user has 0 followers.
The user has 25 friends.


#### Collecting New Data

The new data is collected with the help of api.search function of twitter. 
Search is made based on 20 most similar words based on original work and around 50 tweets for each tweet is queried.

In [7]:
# Function to hit API and writing to Dataframe
def keywordSearch(ls):
    date_since = "2020-01-01"
    newTweets =[]
    for word in ls:
        new_search = word + " -filter:retweets"
        twts = tweepy.Cursor(api.search,
                           q=new_search,
                           lang="en",
                           since=date_since).items(50)
        for t in twts:
            x={}
            x['id'] =  t.id
            x['text'] =  t.text
            newTweets.append(x)
    ndf = pd.DataFrame(newTweets)
    return ndf

Function call with 20 words and the result to be stored in an excel.

In [8]:
ls =["islam","bitches","feminists","trans+woman","gay","dwarf","sluts","","negro","nigga","asians","france+muslim","chinese+virus","terrorist","slave","jews","murderer","blonde","taliban","whites"]
#ls = ["dwarf"]
newTwtsDf =keywordSearch(ls)
newTwtsDf.index += 1 
newTwtsDf.to_csv('newtweets.csv')

In [9]:
newTwtsDf.head()

,id,text
1,1324187087436832768,"@JaskaranS_ Lol, too powerless and irrelevant ..."
2,1324187076733095936,"Islam didn't steal and killing, it invented ""w..."
3,1324186992788164609,@hhhggjjhhg Ashraf will even say it is Islamop...
4,1324186949503066113,idk what is goin to happen in the future but i...
5,1324186945363210242,if you don't know about the rule of Islam then...


**The csv generated above is again further processed in excel for manual annotations.



#### Collecting Orginal Data

Source data can be downloaded from https://github.com/zeerakw/hatespeech. It contains tweet id's and corresponding annotations where tweets are labelled as either Racist, Sexist or Neither Racist or Sexist.

Twitter API(statuses_lookup) was used to retrieve the tweets using given tweet IDs and some basic processing was done as required by the source code.

In [10]:
file_name = 'C:/Users/Owner/Downloads/source4.xlsx'
#df = pd.read_csv('C:/Users/Owner/Downloads/source4.xlsx')
df = pd.read_excel(file_name, sheet_name = "Sheet2")
df.head()

,tweetID,label
0,572342978255048705,racism
1,572341498827522049,racism
2,572340476503724032,racism
3,572334712804384768,racism
4,572332655397629952,racism


In [12]:
racismdf=df[df['label']=="racism"]
sexismdf=df[df['label']=="sexism"]
neitherdf=df[df['label']=="none"]
#neitherdf=df[df['label']=="neither"]

print("Length of Total dataset from source:",len(racismdf) +len(sexismdf)+len(neitherdf),
      "\nLenth of Racism annotated data:",len(racismdf),
      "\nLenth of Sexism annotated data:",len(sexismdf),
      "\nLenth of Neither annotated data:",len(neitherdf))

Length of Total dataset from source: 17005 
Lenth of Racism annotated data: 2068 
Lenth of Sexism annotated data: 3378 
Lenth of Neither annotated data: 11559


Function defined to lookup the tweets for given tweet IDs

In [16]:
def lookup_tweets(tweet_IDs, api):
    full_tweets = []
    tweet_count = len(tweet_IDs)
    try:
        for i in range(int(tweet_count / 100) + 1):
            end_loc = min((i + 1) * 100, tweet_count)
            x =tweet_IDs[i * 100:end_loc]
            full_tweets.extend(
                api.statuses_lookup(tweet_IDs[i * 100:end_loc])
            )
        return full_tweets
    except tweepy.TweepError:
        print('Something went wrong')

In [17]:
def gettweets(daf):
    good_tweet_ids = [i for i in daf.tweetID] #tweet ids to look up 
    results = lookup_tweets(good_tweet_ids, api) #apply function
    return results

In [18]:
#for racism
racism_results = gettweets(racismdf)
print("Total tweet IDs for Racism after lookup is:",len(racism_results),"out of ",len(racismdf))
sexism_results = gettweets(sexismdf)
print("Total tweet IDs for Sexism after lookup is:",len(sexism_results),"out of ",len(sexismdf))
neither_results = gettweets(neitherdf)
print("Total tweet IDs for Neither after lookup is:",len(neither_results),"out of ",len(neitherdf))


Total tweet IDs for Racism after lookup is: 77 out of  2068
Total tweet IDs for Sexism after lookup is: 2777 out of  3378
Total tweet IDs for Neither after lookup is: 7797 out of  11559


Function to write the results to json files as required by the source code.

In [ ]:
def writeResultsToJSON(tweets,screen_name):    
    listOfTweets = []
    for json_tweet in tweets:
        x={}
        t = json_tweet._json
        x['id'] =  t['id']
        x['text'] =  t['text']
        x['user'] ={}
        x['user']['name'] =  t['user']['name']
        x['Annotation'] =  screen_name
        listOfTweets.append(x)
    with open(screen_name +'.json', mode = 'w') as file:
        file.write(json.dumps(listOfTweets, indent = 4))

In [ ]:
#writing to JSON
writeResultsToJSON(racism_results,"racism")  
writeResultsToJSON(sexism_results,"sexism")
writeResultsToJSON(neither_results,"neither")

************************************************End Of Data Collection********************************************************.